# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
housing_data = pd.read_csv('cal_housing_clean.csv')

In [4]:
housing_data.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [5]:
x_data = housing_data.drop(['medianHouseValue'], axis=1)

In [6]:
labels = housing_data['medianHouseValue']

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x_data,
                                                    labels,
                                                    test_size=0.3, 
                                                    random_state=101)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [8]:
import sklearn.preprocessing as prep

In [9]:
scaler = prep.MinMaxScaler()

In [10]:
x_train = pd.DataFrame(data=scaler.fit_transform(x_train), columns = x_train.columns, index=x_train.index)
x_test = pd.DataFrame(data=scaler.fit_transform(x_test), columns = x_test.columns, index=x_test.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [11]:
import tensorflow as tf

In [12]:
features = []

for column in x_train.columns:
    feature = tf.feature_column.numeric_column(column)
    features.append(feature)

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [13]:
input_func = tf.estimator.inputs.pandas_input_fn(x=x_train, y=y_train, batch_size=10, num_epochs=1000, shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [54]:
dnn_estimator = tf.estimator.DNNRegressor(hidden_units=[8,8,8], feature_columns=features)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_session_config': None, '_log_step_count_steps': 100, '_tf_random_seed': 1, '_save_checkpoints_secs': 600, '_model_dir': 'C:\\Users\\CASSIO~1.MAC\\AppData\\Local\\Temp\\tmp_2p2c30s'}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [73]:
dnn_estimator.train(input_fn=input_func, steps=8000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from C:\Users\CASSIO~1.MAC\AppData\Local\Temp\tmp_2p2c30s\model.ckpt-6000
INFO:tensorflow:Saving checkpoints for 6001 into C:\Users\CASSIO~1.MAC\AppData\Local\Temp\tmp_2p2c30s\model.ckpt.
INFO:tensorflow:loss = 1.28097e+11, step = 6001
INFO:tensorflow:global_step/sec: 657.891
INFO:tensorflow:loss = 1.21652e+11, step = 6101 (0.154 sec)
INFO:tensorflow:global_step/sec: 694.411
INFO:tensorflow:loss = 9.21643e+10, step = 6201 (0.144 sec)
INFO:tensorflow:global_step/sec: 662.289
INFO:tensorflow:loss = 9.38719e+10, step = 6301 (0.151 sec)
INFO:tensorflow:global_step/sec: 709.221
INFO:tensorflow:loss = 3.63872e+10, step = 6401 (0.141 sec)
INFO:tensorflow:global_step/sec: 740.747
INFO:tensorflow:loss = 6.3039e+10, step = 6501 (0.135 sec)
INFO:tensorflow:global_step/sec: 671.141
INFO:tensorflow:loss = 9.93162e+10, step = 6601 (0.149 sec)
INFO:tensorflow:global_step/sec: 636.947
INFO:tensorflow:loss = 9.61038e+10, s

INFO:tensorflow:global_step/sec: 689.661
INFO:tensorflow:loss = 1.8321e+11, step = 13901 (0.145 sec)
INFO:tensorflow:Saving checkpoints for 14000 into C:\Users\CASSIO~1.MAC\AppData\Local\Temp\tmp_2p2c30s\model.ckpt.
INFO:tensorflow:Loss for final step: 1.50216e+11.


** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [74]:
prediction_input_func = tf.estimator.inputs.pandas_input_fn(x=x_test, 
                                                            batch_size=10,
                                                            num_epochs=1,
                                                            shuffle=False)

In [75]:
pred_gen = dnn_estimator.predict(prediction_input_func)

In [76]:
predictions = list(pred_gen)

INFO:tensorflow:Restoring parameters from C:\Users\CASSIO~1.MAC\AppData\Local\Temp\tmp_2p2c30s\model.ckpt-14000


In [77]:
predictions

[{'predictions': array([ 240177.15625], dtype=float32)},
 {'predictions': array([ 307587.78125], dtype=float32)},
 {'predictions': array([ 218650.015625], dtype=float32)},
 {'predictions': array([ 190192.78125], dtype=float32)},
 {'predictions': array([ 281145.09375], dtype=float32)},
 {'predictions': array([ 210167.015625], dtype=float32)},
 {'predictions': array([ 232858.625], dtype=float32)},
 {'predictions': array([ 211637.078125], dtype=float32)},
 {'predictions': array([ 222550.09375], dtype=float32)},
 {'predictions': array([ 191979.765625], dtype=float32)},
 {'predictions': array([ 213292.1875], dtype=float32)},
 {'predictions': array([ 238381.734375], dtype=float32)},
 {'predictions': array([ 197889.], dtype=float32)},
 {'predictions': array([ 181855.765625], dtype=float32)},
 {'predictions': array([ 266672.96875], dtype=float32)},
 {'predictions': array([ 180325.046875], dtype=float32)},
 {'predictions': array([ 203596.625], dtype=float32)},
 {'predictions': array([ 194612.15

** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [78]:
from sklearn.metrics import mean_squared_error

In [79]:
mean_squared_error(y_test, list(map(lambda x: x['predictions'][0], predictions)))**0.5

100387.43402207189

# Great Job!